Mount to MyDrive:

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Download repo and install dependencies:

In [ ]:
import os

%cd /content/gdrive/MyDrive

if os.getcwd() == '/content/gdrive/MyDrive/Book-Recommendation-System':
  %cd ..
  !rm -r ./NLP-Business-News-Summarization/

# Remove Colab default sample_data
if os.path.exists('/content/sample_data'):
  !rm -r /content/sample_data

repo_name = "Book-Recommendation-System" # Repository Name

if not os.path.isdir('./' + repo_name):
  # Clone GitHub Files
  git_path = 'https://github.com/christianwhollar/Book-Recommendation-System.git'
  !git clone "{git_path}"

# Change to new directory
if not os.path.basename(os.getcwd()) == repo_name:
  %cd "{repo_name}"

Running setup.py will do the following:
* Download the Goodreads & Book Crossing .csv data
* Merge the data & prepare for modeling
* Train the HybridRecommendation model
* Evaluate the HybridRecommendation model
* Train the ContentFilter model
* Evaluate the Content Filer model

In [ ]:
%run scripts/setup.py

Launch the streamlit app:

In [ ]:
!streamlit run app.py